In [8]:
# The code was removed by DSX for sharing.

In [2]:
class hn_collector():
    
    import requests
    from bs4 import BeautifulSoup
    
    def __init__(self):
        self.API_BASE = 'https://hacker-news.firebaseio.com/v0/'
        self.TOP ='topstories.json'
        self.NEW ='newstories.json'
        self.ITEM ='item/'
        self.HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'}
        
    def get_top_stories(self):
        return(eval(requests.get(self.API_BASE + self.TOP).content))
    
    def get_new_stories(self):
        return(eval(requests.get(self.API_BASE + self.NEW).content))
    
    def get_story(self, story_id):
        try:
            return(eval(requests.get(self.API_BASE + self.ITEM + str(story_id)+".json").content))
        except:
            pass
    
    def get_text(self, story_url):

        try:
            page = requests.get(story_url, headers=self.HEADERS)
            soup = BeautifulSoup(page.content,'lxml')
            s= ' '.join([p.text for p in soup.find_all('p')])
            return(s)
        except:
            return(None)
        

In [4]:
hn = hn_collector() #instantiate class then use methods to initiate process

CURRENT_NEW = 'current_new.json'
CURRENT_TOP = 'current_top.json'

In [98]:
story_list = []
NEW_STORIES = hn.get_new_stories()
cos.download_file(Bucket=credentials['BUCKET'],Key='new_stories.json',Filename=CURRENT_NEW)

with open(CURRENT_NEW) as json_data:
    d = json.load(json_data)
    
df = json_normalize(d).drop_duplicates('url',keep='first')
diff_stories = set(NEW_STORIES) - set(df.id)
for story in list(diff_stories)[:100]:
    try:
        s = hn.get_story(story)
        raw_text = hn.get_text(s.get('url',''))
        s['kids'] = str(s.get('kids',None))
        s['text'] = raw_text
        story_list.append(s)
    except Exception as e:
        print(e)

if len(story_list) > 0:
    samp = pd.DataFrame(story_list).drop_duplicates('id').sort_values('time',ascending=False)
    samp = samp[samp.text.str.len() > 200][:100]
    samp.to_json(CURRENT_NEW,orient='records')
    cos.upload_file(Filename='new_stories.json',Bucket=credentials['BUCKET'],Key=CURRENT_NEW)

In [60]:
story_list = []
TOP_STORIES = hn.get_top_stories()
cos.download_file(Bucket=credentials['BUCKET'],Key=CURRENT_TOP,Filename=CURRENT_TOP)

with open(CURRENT_TOP) as json_data:
    d = json.load(json_data)
    
df = json_normalize(d).drop_duplicates('url',keep='first')
diff_stories = set(TOP_STORIES) - set(df.id)

for story in list(diff_stories)[:100]:
    try:
        s = hn.get_story(story)
        raw_text = hn.get_text(s.get('url',''))
        s['kids'] = str(s.get('kids',None))
        s['text'] = raw_text
        story_list.append(s)
    except Exception as e:
        print(e)

if len(story_list) > 0:
    samp = pd.DataFrame(story_list).drop_duplicates('id').sort_values('time',ascending=False)
    samp = samp[samp.text.str.len() > 200][:100]
    samp.to_json(CURRENT_TOP,orient='records')
    cos.upload_file(Filename=CURRENT_TOP,Bucket=credentials['BUCKET'],Key=CURRENT_TOP)

393
